# U.S. Medical Insurance Costs

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
# ---------------------------------

import duckdb
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
# %sql duckdb:///:memory:
# # %sql duckdb:///path/to/file.db

# import pandas_ml as pdml
import sklearn.datasets as datasets
import geopandas

from jupyter_datatables import init_datatables_mode
# init_datatables_mode()

from summarytools import dfSummary, tabset
import sidetable
import sketch

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [3]:
raw_data = pd.DataFrame(pd.read_csv("insurance.csv"))

In [18]:
raw_data.describe(include="all")

,age,sex,bmi,children,smoker,region,charges
count,1338.000000,1338,1338.000000,1338.000000,1338,1338,1338.000000
unique,NaN,2,NaN,NaN,2,4,NaN
top,NaN,male,NaN,NaN,no,southeast,NaN
freq,NaN,676,NaN,NaN,1064,364,NaN
mean,39.207025,NaN,30.663397,1.094918,NaN,NaN,13270.422265
std,14.049960,NaN,6.098187,1.205493,NaN,NaN,12110.011237
min,18.000000,NaN,15.960000,0.000000,NaN,NaN,1121.873900
25%,27.000000,NaN,26.296250,0.000000,NaN,NaN,4740.287150
50%,39.000000,NaN,30.400000,1.000000,NaN,NaN,9382.033000
75%,51.000000,NaN,34.693750,2.000000,NaN,NaN,16639.912515


In [5]:
dfSummary(raw_data, is_collapsible = True)
# tabset({
#     'titanic': dfSummary(titanic).render(),
#     'vaccine': dfSummary(vaccine).render()
#     })


No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,age[int64],Mean (sd) : 39.2 (14.0)min < med < max:18.0 < 39.0 < 64.0IQR (CV) : 24.0 (2.8),47 distinct values,,0(0.0%)
2,sex[object],1. male2. female,676 (50.5%)662 (49.5%),,0(0.0%)
3,bmi[float64],Mean (sd) : 30.7 (6.1)min < med < max:16.0 < 30.4 < 53.1IQR (CV) : 8.4 (5.0),548 distinct values,,0(0.0%)
4,children[int64],1. 02. 13. 24. 35. 46. 5,574 (42.9%)324 (24.2%)240 (17.9%)157 (11.7%)25 (1.9%)18 (1.3%),,0(0.0%)
5,smoker[object],1. no2. yes,"1,064 (79.5%)274 (20.5%)",,0(0.0%)
6,region[object],1. southeast2. southwest3. northwest4. northeast,364 (27.2%)325 (24.3%)325 (24.3%)324 (24.2%),,0(0.0%)
7,charges[float64],Mean (sd) : 13270.4 (12110.0)min < med < max:1121.9 < 9382.0 < 63770.4IQR (CV) : 11899.6 (1.1),"1,337 distinct values",,0(0.0%)


In [6]:
raw_data.stb.freq(["children"])

,children,count,percent,cumulative_count,cumulative_percent
0,0,574,42.899851,574,42.899851
1,1,324,24.215247,898,67.115097
2,2,240,17.937220,1138,85.052317
3,3,157,11.733931,1295,96.786248
4,4,25,1.868460,1320,98.654709
5,5,18,1.345291,1338,100.000000


In [7]:
raw_data.stb.missing()

,missing,total,percent
age,0,1338,0.0
sex,0,1338,0.0
bmi,0,1338,0.0
children,0,1338,0.0
smoker,0,1338,0.0
region,0,1338,0.0
charges,0,1338,0.0


In [33]:
raw_data.groupby(["smoker", "sex"]).agg({"charges": ["mean", "min", "max"]}).stb.subtotal()

charges                          
                                    mean         min           max
smoker      sex                                                   
no          female           8762.297300   1607.5101   36910.60803
            male             8087.204731   1121.8739   32108.66282
            no - subtotal   16849.502031   2729.3840   69019.27085
yes         female          30678.996276  13844.5060   63770.42801
            male            33042.005975  12829.4551   62592.87309
            yes - subtotal  63721.002252  26673.9611  126363.30110
grand_total                 80570.504282  29403.3451  195382.57195

In [9]:
raw_data.sketch.ask("Which columns are integer type?")


In [10]:
raw_data.sketch.howto("Plot the charges versus sex")

In [17]:
# Find out the average age of the patients in the dataset.
def avgAge(df):
    sumAge = 0;
    for i in range(len(df)):
        sumAge += df["age"][i]
    return sumAge/len(df)

avgAge(raw_data)

39.20702541106129

In [31]:
# Analyze where a majority of the individuals are from.
def peopleRegion(df):
    regionsCount = {}
    regionsPercent = {}
    for i in range(len(df)):
        currRegion = df["region"][i]
        if currRegion not in regionsCount:
            regionsCount[currRegion] = 1
        else:
            regionsCount[currRegion] += 1

    for region in regionsCount:
        regionsPercent[region] = "{:.2%}".format(regionsCount[region] / len(df))

    return regionsCount, regionsPercent

peopleRegion(raw_data)

({'southwest': 325, 'southeast': 364, 'northwest': 325, 'northeast': 324},
 {'southwest': '24.29%',
  'southeast': '27.20%',
  'northwest': '24.29%',
  'northeast': '24.22%'})

In [53]:
# Look at the different costs between smokers vs. non-smokers.
def avgSmokerCost(df):
    sumCost = {}
    avgCost = {}
    countSmokers = {}
    for i in range(len(df)):
        currSmoker = df["smoker"][i]
        smokerCost = df["charges"][i]
        
        if currSmoker not in sumCost:
            sumCost[currSmoker] = smokerCost
            countSmokers[currSmoker] = 1
        else:
            sumCost[currSmoker] += smokerCost
            countSmokers[currSmoker] += 1

    for cost in sumCost:
        avgCost[cost] = sumCost[cost] / countSmokers[cost]

    return avgCost

avgSmokerCost(raw_data)

{'yes': 32050.23183153285, 'no': 8434.268297856199}

In [ ]:
# Figure out what the average age is for someone who has at least one child in this dataset.
